In [2]:
pip install selenium undetected-chromedriver pandas tqdm


Note: you may need to restart the kernel to use updated packages.


In [11]:
!pip install webdriver-manager

In [28]:
import os
import re
import time
import pickle
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Paths
cookies_path = '../notebooks/cookies'
urls_path = '../data/raw/review_urls.txt'
output_csv = '../data/raw/amazon_reviews_raw.csv'

# Load cookies
def load_cookies(driver, cookies_path):
    for file in os.listdir(cookies_path):
        if file.endswith('.pkl'):
            cookies = pickle.load(open(os.path.join(cookies_path, file), "rb"))
            for cookie in cookies:
                if 'sameSite' in cookie and cookie['sameSite'] == 'None':
                    cookie['sameSite'] = 'Strict'
                driver.add_cookie(cookie)

# Scroll to load all dynamic reviews
def scroll_to_load_reviews(driver, pause=2):
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(8):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Extract review texts only from current page
def extract_reviews_on_page(driver):
    reviews = []
    review_elements = driver.find_elements(By.CSS_SELECTOR, ".review")
    for r in review_elements:
        try:
            text = r.find_element(By.CSS_SELECTOR, ".review-text-content span").text.strip()
            reviews.append(text)
        except:
            continue
    return reviews

# Click next page if exists
def click_next_page(driver):
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "li.a-last a")
        driver.execute_script("arguments[0].scrollIntoView();", next_button)
        time.sleep(1.5)
        next_button.click()
        return True
    except:
        return False

# Extract overall product rating
def extract_overall_product_rating(driver):
    try:
        rating_element = driver.find_element(By.CSS_SELECTOR, 'span[data-hook="rating-out-of-text"]')
        rating_text = rating_element.text.strip()
        match = re.search(r'([\d.]+)', rating_text)
        return float(match.group(1)) if match else ""
    except:
        return ""

# Extract product ID from URL
def extract_product_id(url):
    match = re.search(r'/product-reviews/([A-Z0-9]{10})', url)
    return match.group(1) if match else None

# Get already scraped productIDs
def get_completed_product_ids(output_csv):
    if os.path.exists(output_csv) and os.path.getsize(output_csv) > 0:
        try:
            df = pd.read_csv(output_csv)
            if "productID" in df.columns:
                return set(df["productID"].dropna().astype(str).unique())
        except Exception as e:
            print(f"⚠️ Error reading existing CSV: {e}")
    return set()

# Scrape reviews
def scrape_amazon_reviews(driver, product_urls, max_reviews_per_product=500):
    completed_ids = get_completed_product_ids(output_csv)

    for url in tqdm(product_urls, desc="Scraping Products"):
        product_id = extract_product_id(url)
        if not product_id or product_id in completed_ids:
            continue

        driver.get(url)
        time.sleep(3)

        overall_rating = extract_overall_product_rating(driver)
        reviews_collected = []

        while len(reviews_collected) < max_reviews_per_product:
            scroll_to_load_reviews(driver)
            page_reviews = extract_reviews_on_page(driver)

            for text in page_reviews:
                if text and len(reviews_collected) < max_reviews_per_product:
                    reviews_collected.append(text)

            if not click_next_page(driver):
                break
            time.sleep(3)

        # Prepare row with review1...review500
        row = {
            "productID": product_id,
            "overall_rating": overall_rating
        }

        for i in range(max_reviews_per_product):
            row[f"review{i+1}"] = reviews_collected[i] if i < len(reviews_collected) else ""

        # Safe CSV save
        try:
            df_existing = pd.read_csv(output_csv) if os.path.exists(output_csv) and os.path.getsize(output_csv) > 0 else pd.DataFrame()
        except Exception as e:
            print(f"⚠️ Could not load existing CSV: {e}")
            df_existing = pd.DataFrame()

        df_new = pd.DataFrame([row])
        df_combined = pd.concat([df_existing, df_new], ignore_index=True)
        df_combined.to_csv(output_csv, index=False)

        print(f"✅ Reviews and rating for {product_id} saved.")

# Setup Selenium
options = Options()
options.add_argument('--start-maximized')
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Inject cookies
driver.get("https://www.amazon.in")
time.sleep(3)
load_cookies(driver, cookies_path)
driver.refresh()
time.sleep(5)
input("🔐 Check if logged in. Press Enter to continue... ")

# Load URLs
with open(urls_path, 'r') as f:
    all_urls = [line.strip() for line in f if line.strip()]

# Test mode control
test_mode = True
num_test_products = 57
product_urls = all_urls[:num_test_products] if test_mode else all_urls

# Start scraper
scrape_amazon_reviews(driver, product_urls)
driver.quit()


🔐 Check if logged in. Press Enter to continue...  


Scraping Products:   2%|█                                                             | 1/57 [01:09<1:04:29, 69.09s/it]

✅ Reviews and rating for B0DSG8SNXH saved.


Scraping Products:   4%|██▏                                                           | 2/57 [02:17<1:03:09, 68.90s/it]

✅ Reviews and rating for B0DFY3XCB6 saved.


...

✅ Reviews and rating for B09ZPL5VYM saved.


Scraping Products:  98%|███████████████████████████████████████████████████████████▉ | 56/57 [1:04:19<01:09, 69.27s/it]

✅ Reviews and rating for B0CHX3TW6X saved.


Scraping Products: 100%|█████████████████████████████████████████████████████████████| 57/57 [1:05:29<00:00, 68.93s/it]

✅ Reviews and rating for B0DH8BZ7V9 saved.
